In [ ]:
import csv

class MazeGenerator:
    def __init__(self, size):
        self.size = size
        # Grid format: (x, y) -> {'N': 1, 'E': 1, 'S': 1, 'W': 1} (All walls initially)
        self.grid = {}
        for x in range(size):
            for y in range(size):
                self.grid[(x, y)] = {'N': 1, 'E': 1, 'S': 1, 'W': 1}

    def open_wall(self, c1, c2):
        x1, y1 = c1
        x2, y2 = c2
        
        # Check bounds
        if not (0 <= x1 < self.size and 0 <= y1 < self.size): return
        if not (0 <= x2 < self.size and 0 <= y2 < self.size): return

        # Determine direction
        if x2 == x1 and y2 == y1 + 1: # c2 is North of c1
            self.grid[c1]['N'] = 0
            self.grid[c2]['S'] = 0
        elif x2 == x1 and y2 == y1 - 1: # c2 is South of c1
            self.grid[c1]['S'] = 0
            self.grid[c2]['N'] = 0
        elif x2 == x1 + 1 and y2 == y1: # c2 is East of c1
            self.grid[c1]['E'] = 0
            self.grid[c2]['W'] = 0
        elif x2 == x1 - 1 and y2 == y1: # c2 is West of c1
            self.grid[c1]['W'] = 0
            self.grid[c2]['E'] = 0

    def generate(self):
        # Number of concentric rings
        num_rings = self.size // 4
        
        for k in range(num_rings):
            min_bound = 2 * k
            max_bound = self.size - 1 - (2 * k)
            
            # --- 1. Bottom Strip (Left to Right) ---
            current_x = min_bound
            while current_x < max_bound:
                if (current_x - min_bound) % 2 == 0: # Up Stroke
                    next_y = min_bound + 1
                    self.open_wall((current_x, min_bound), (current_x, next_y))
                    if current_x < max_bound - 1:
                        self.open_wall((current_x, next_y), (current_x + 1, next_y))
                else: # Down Stroke
                    next_y = min_bound
                    self.open_wall((current_x, min_bound + 1), (current_x, next_y))
                    if current_x < max_bound - 1:
                        self.open_wall((current_x, next_y), (current_x + 1, next_y))
                current_x += 1
            
            # Corner Sequence Bottom-Right
            self.open_wall((max_bound - 1, min_bound), (max_bound, min_bound))
            self.open_wall((max_bound, min_bound), (max_bound, min_bound + 1))
            self.open_wall((max_bound, min_bound + 1), (max_bound - 1, min_bound + 1))
            self.open_wall((max_bound - 1, min_bound + 1), (max_bound - 1, min_bound + 2))

            # --- 2. Right Strip (Bottom to Top) ---
            current_y = min_bound + 2
            while current_y < max_bound:
                rel_y = current_y - (min_bound + 2)
                if rel_y % 2 == 0: # Right Stroke
                    self.open_wall((max_bound - 1, current_y), (max_bound, current_y))
                    if current_y < max_bound - 1:
                        self.open_wall((max_bound, current_y), (max_bound, current_y + 1))
                else: # Left Stroke
                    self.open_wall((max_bound, current_y), (max_bound - 1, current_y))
                    if current_y < max_bound - 1:
                        self.open_wall((max_bound - 1, current_y), (max_bound - 1, current_y + 1))
                current_y += 1
            
            # Corner Sequence Top-Right
            self.open_wall((max_bound, max_bound - 1), (max_bound, max_bound))
            self.open_wall((max_bound, max_bound), (max_bound - 1, max_bound))
            self.open_wall((max_bound - 1, max_bound), (max_bound - 2, max_bound))

            # --- 3. Top Strip (Right to Left) ---
            current_x = max_bound - 2
            while current_x > min_bound + 1:
                rel_x = (max_bound - 2) - current_x
                if rel_x % 2 == 0: # Down Stroke
                    self.open_wall((current_x, max_bound), (current_x, max_bound - 1))
                    if current_x > min_bound + 1:
                        self.open_wall((current_x, max_bound - 1), (current_x - 1, max_bound - 1))
                else: # Up Stroke
                    self.open_wall((current_x, max_bound - 1), (current_x, max_bound))
                    if current_x > min_bound + 1:
                        self.open_wall((current_x, max_bound), (current_x - 1, max_bound))
                current_x -= 1
            
            # Corner Sequence Top-Left
            self.open_wall((min_bound + 1, max_bound), (min_bound, max_bound))
            self.open_wall((min_bound, max_bound), (min_bound, max_bound - 1))
            self.open_wall((min_bound, max_bound - 1), (min_bound + 1, max_bound - 1))
            self.open_wall((min_bound + 1, max_bound - 1), (min_bound + 1, max_bound - 2))

            # --- 4. Left Strip (Top to Bottom) ---
            current_y = max_bound - 2
            # We go down to min_bound + 2 to meet the transition point
            while current_y >= min_bound + 2:
                rel_y = (max_bound - 2) - current_y
                if rel_y % 2 == 0: # Left Stroke
                    self.open_wall((min_bound + 1, current_y), (min_bound, current_y))
                    if current_y > min_bound + 2:
                        self.open_wall((min_bound, current_y), (min_bound, current_y - 1))
                else: # Right Stroke
                    self.open_wall((min_bound, current_y), (min_bound + 1, current_y))
                    if current_y > min_bound + 2:
                        self.open_wall((min_bound + 1, current_y), (min_bound + 1, current_y - 1))
                current_y -= 1
        
        # --- 5. Apply Specific Fixes (The "Misplaced Wall" Fix) ---
        # User requested removing right wall at (0, 2), (2, 4), (4, 6)...
        # Pattern: (2k, 2k+2) connect to (2k+1, 2k+2)
        for k in range(num_rings):
            fix_x = 2 * k
            fix_y = 2 * k + 2
            # Open the East wall (Right wall)
            self.open_wall((fix_x, fix_y), (fix_x + 1, fix_y))
            
            # Also ensure connection to the next ring's start if not the very center
            # The next ring starts at (2k+2, 2k+2). 
            # Usually the flow is (2k, 2k+2) -> (2k+1, 2k+2) -> (2k+2, 2k+2)
            if k < num_rings - 1:
                 self.open_wall((fix_x + 1, fix_y), (fix_x + 2, fix_y))

        # --- 6. Center Goal Area ---
        c_min = self.size // 2 - 1
        c_max = self.size // 2
        # Open 2x2 center
        self.open_wall((c_min, c_min), (c_max, c_min))
        self.open_wall((c_min, c_min), (c_min, c_max))
        self.open_wall((c_max, c_max), (c_max, c_min))
        self.open_wall((c_max, c_max), (c_min, c_max))

    def to_string(self):
        lines = []
        # Sort by x then y
        for x in range(self.size):
            for y in range(self.size):
                c = self.grid[(x, y)]
                lines.append(f"{x} {y} {c['N']} {c['E']} {c['S']} {c['W']}")
        return "\n".join(lines)

def generate_maze_csv_32():
    SIZE = 32
    MAZE_ID = 0
    START = "(0, 0)"
    g1 = SIZE // 2 - 1
    g2 = SIZE // 2
    GOALS = f"[({g1},{g1}),({g1},{g2}),({g2},{g1}),({g2},{g2})]"
    
    gen = MazeGenerator(SIZE)
    gen.generate()
    maze_string = gen.to_string()
    
    filename = 'maze_32x32.csv'
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['id', 'size', 'start', 'goals', 'maze'])
        writer.writerow([MAZE_ID, SIZE, START, GOALS, maze_string])
    
    print(f"Generated {filename}")

if __name__ == "__main__":
    generate_maze_csv_32()

Generated maze_32x32.csv


In [1]:
import csv

class MazeGenerator:
    def __init__(self, size):
        self.size = size
        # Grid format: (x, y) -> {'N': 1, 'E': 1, 'S': 1, 'W': 1} (All walls initially)
        self.grid = {}
        for x in range(size):
            for y in range(size):
                self.grid[(x, y)] = {'N': 1, 'E': 1, 'S': 1, 'W': 1}

    def open_wall(self, c1, c2):
        """Removes the wall between cell1 and cell2."""
        x1, y1 = c1
        x2, y2 = c2
        
        # Check bounds
        if not (0 <= x1 < self.size and 0 <= y1 < self.size): return
        if not (0 <= x2 < self.size and 0 <= y2 < self.size): return

        # Determine direction and remove wall from both sides
        if x2 == x1 and y2 == y1 + 1: # c2 is North of c1
            self.grid[c1]['N'] = 0
            self.grid[c2]['S'] = 0
        elif x2 == x1 and y2 == y1 - 1: # c2 is South of c1
            self.grid[c1]['S'] = 0
            self.grid[c2]['N'] = 0
        elif x2 == x1 + 1 and y2 == y1: # c2 is East of c1
            self.grid[c1]['E'] = 0
            self.grid[c2]['W'] = 0
        elif x2 == x1 - 1 and y2 == y1: # c2 is West of c1
            self.grid[c1]['W'] = 0
            self.grid[c2]['E'] = 0

    def generate_base_spiral(self):
        """Generates the 32x32 Spiral Zigzag pattern."""
        num_rings = self.size // 4
        for k in range(num_rings):
            min_bound = 2 * k
            max_bound = self.size - 1 - (2 * k)
            
            # Bottom Strip
            current_x = min_bound
            while current_x < max_bound:
                if (current_x - min_bound) % 2 == 0:
                    self.open_wall((current_x, min_bound), (current_x, min_bound + 1))
                    if current_x < max_bound - 1:
                        self.open_wall((current_x, min_bound + 1), (current_x + 1, min_bound + 1))
                else:
                    self.open_wall((current_x, min_bound + 1), (current_x, min_bound))
                    if current_x < max_bound - 1:
                        self.open_wall((current_x, min_bound), (current_x + 1, min_bound))
                current_x += 1
            
            # Corner BR
            self.open_wall((max_bound - 1, min_bound), (max_bound, min_bound))
            self.open_wall((max_bound, min_bound), (max_bound, min_bound + 1))
            self.open_wall((max_bound, min_bound + 1), (max_bound - 1, min_bound + 1))
            self.open_wall((max_bound - 1, min_bound + 1), (max_bound - 1, min_bound + 2))

            # Right Strip
            current_y = min_bound + 2
            while current_y < max_bound:
                if (current_y - (min_bound + 2)) % 2 == 0:
                    self.open_wall((max_bound - 1, current_y), (max_bound, current_y))
                    if current_y < max_bound - 1:
                        self.open_wall((max_bound, current_y), (max_bound, current_y + 1))
                else:
                    self.open_wall((max_bound, current_y), (max_bound - 1, current_y))
                    if current_y < max_bound - 1:
                        self.open_wall((max_bound - 1, current_y), (max_bound - 1, current_y + 1))
                current_y += 1
            
            # Corner TR
            self.open_wall((max_bound, max_bound - 1), (max_bound, max_bound))
            self.open_wall((max_bound, max_bound), (max_bound - 1, max_bound))
            self.open_wall((max_bound - 1, max_bound), (max_bound - 2, max_bound))

            # Top Strip
            current_x = max_bound - 2
            while current_x > min_bound + 1:
                if ((max_bound - 2) - current_x) % 2 == 0:
                    self.open_wall((current_x, max_bound), (current_x, max_bound - 1))
                    if current_x > min_bound + 1:
                        self.open_wall((current_x, max_bound - 1), (current_x - 1, max_bound - 1))
                else:
                    self.open_wall((current_x, max_bound - 1), (current_x, max_bound))
                    if current_x > min_bound + 1:
                        self.open_wall((current_x, max_bound), (current_x - 1, max_bound))
                current_x -= 1
            
            # Corner TL
            self.open_wall((min_bound + 1, max_bound), (min_bound, max_bound))
            self.open_wall((min_bound, max_bound), (min_bound, max_bound - 1))
            self.open_wall((min_bound, max_bound - 1), (min_bound + 1, max_bound - 1))
            self.open_wall((min_bound + 1, max_bound - 1), (min_bound + 1, max_bound - 2))

            # Left Strip
            current_y = max_bound - 2
            while current_y >= min_bound + 2:
                if ((max_bound - 2) - current_y) % 2 == 0:
                    self.open_wall((min_bound + 1, current_y), (min_bound, current_y))
                    if current_y > min_bound + 2:
                        self.open_wall((min_bound, current_y), (min_bound, current_y - 1))
                else:
                    self.open_wall((min_bound, current_y), (min_bound + 1, current_y))
                    if current_y > min_bound + 2:
                        self.open_wall((min_bound + 1, current_y), (min_bound + 1, current_y - 1))
                current_y -= 1
        
        # Apply Fixes
        for k in range(num_rings):
            fix_x, fix_y = 2 * k, 2 * k + 2
            self.open_wall((fix_x, fix_y), (fix_x + 1, fix_y))
            if k < num_rings - 1:
                 self.open_wall((fix_x + 1, fix_y), (fix_x + 2, fix_y))

    def get_total_wall_count(self):
        total_flags = 0
        for coord in self.grid:
            cell = self.grid[coord]
            total_flags += (cell['N'] + cell['E'] + cell['S'] + cell['W'])
        return total_flags / 2

    def process_cell_one_by_one(self, cell, x_min, x_max, y_min, y_max, target_walls):
        """
        Removes walls of a specific cell IF they connect to the interior of the current rectangle.
        Keeps walls that form the outer boundary of the current rectangle.
        Returns True if we should stop (target met), False otherwise.
        """
        cx, cy = cell
        current_walls = self.get_total_wall_count()
        if current_walls <= target_walls: return True

        # Try to open West (if not left boundary)
        if cx > x_min: self.open_wall((cx, cy), (cx - 1, cy))
        
        # Try to open East (if not right boundary)
        if cx < x_max: self.open_wall((cx, cy), (cx + 1, cy))
        
        # Try to open South (if not bottom boundary)
        if cy > y_min: self.open_wall((cx, cy), (cx, cy - 1))
        
        # Try to open North (if not top boundary)
        if cy < y_max: self.open_wall((cx, cy), (cx, cy + 1))

        # Check immediately
        if self.get_total_wall_count() <= target_walls:
            return True
        return False

    def reduce_walls_iteratively(self, target_walls):
        # 1. Define initial Center 2x2
        x_min, x_max = 15, 16
        y_min, y_max = 15, 16
        
        # List of initial cells (just process them)
        initial_cells = [(15, 16), (16, 16), (16, 15), (15, 15)]
        
        for c in initial_cells:
            if self.process_cell_one_by_one(c, x_min, x_max, y_min, y_max, target_walls):
                print(f"Target met at initial center block.")
                return

        # 2. Expansion Sequence
        # Pattern: X, Y, X, X, Y, Y, X, X, Y, Y...
        steps = ['X', 'Y']
        for _ in range(10): steps.extend(['X', 'X', 'Y', 'Y'])
        
        print(f"Starting expansion from center ({x_min}-{x_max}, {y_min}-{y_max}). Target: {target_walls}")

        for axis in steps:
            prev_xmin, prev_xmax = x_min, x_max
            prev_ymin, prev_ymax = y_min, y_max
            
            # Calculate New Bounds
            if axis == 'X':
                if x_min > 0: x_min -= 1
                if x_max < self.size - 1: x_max += 1
            elif axis == 'Y':
                if y_min > 0: y_min -= 1
                if y_max < self.size - 1: y_max += 1
            
            # Identify New Cells (Perimeter)
            # We want a Clockwise traversal starting from Top-Left (x_min, y_max)
            new_cells = []
            
            # 1. Top Edge (Left to Right) -> y = y_max
            for x in range(x_min, x_max + 1):
                if (x, y_max) not in new_cells and (y_max > prev_ymax or x < prev_xmin or x > prev_xmax):
                    new_cells.append((x, y_max))
            
            # 2. Right Edge (Top to Bottom) -> x = x_max
            for y in range(y_max - 1, y_min - 1, -1):
                if (x_max, y) not in new_cells and (x_max > prev_xmax or y < prev_ymin or y > prev_ymax):
                    new_cells.append((x_max, y))
            
            # 3. Bottom Edge (Right to Left) -> y = y_min
            for x in range(x_max - 1, x_min - 1, -1):
                if (x, y_min) not in new_cells and (y_min < prev_ymin or x < prev_xmin or x > prev_xmax):
                    new_cells.append((x, y_min))
            
            # 4. Left Edge (Bottom to Top) -> x = x_min
            for y in range(y_min + 1, y_max):
                if (x_min, y) not in new_cells and (x_min < prev_xmin or y < prev_ymin or y > prev_ymax):
                    new_cells.append((x_min, y))

            # Process cells one by one
            for cell in new_cells:
                stop = self.process_cell_one_by_one(cell, x_min, x_max, y_min, y_max, target_walls)
                if stop:
                    print(f"Target reached at cell {cell} during Expansion {axis}.")
                    return
            
            # Log progress
            current = self.get_total_wall_count()
            print(f"Expanded {axis} to ({x_min},{y_min})-({x_max},{y_max}). Remaining: {int(current)}")
            if current <= target_walls:
                return

    def to_string(self):
        lines = []
        for x in range(self.size):
            for y in range(self.size):
                c = self.grid[(x, y)]
                lines.append(f"{x} {y} {c['N']} {c['E']} {c['S']} {c['W']}")
        return "\n".join(lines)

def generate_maze_csv_32(target_walls):
    SIZE = 32
    MAZE_ID = 0
    START = "(0, 0)"
    g1 = SIZE // 2 - 1
    g2 = SIZE // 2
    GOALS = f"[({g1},{g1}),({g1},{g2}),({g2},{g1}),({g2},{g2})]"
    
    gen = MazeGenerator(SIZE)
    gen.generate_base_spiral()
    gen.reduce_walls_iteratively(target_walls)
    
    maze_string = gen.to_string()
    filename = 'maze_32x32_iterative.csv'
    
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['id', 'size', 'start', 'goals', 'maze'])
        writer.writerow([MAZE_ID, SIZE, START, GOALS, maze_string])
    
    print(f"Successfully generated {filename}")

if __name__ == "__main__":
    # --- INPUT ---
    # Example: Student ID sum 2600 -> Target 1300
    # The code will remove walls one-by-one spiraling out from center 
    # until this number is reached.
    TARGET_WALL_COUNT = 700
    
    generate_maze_csv_32(TARGET_WALL_COUNT)

Starting expansion from center (15-16, 15-16). Target: 700
Expanded X to (14,15)-(17,16). Remaining: 1012
Expanded Y to (14,14)-(17,17). Remaining: 1009
Expanded X to (13,14)-(18,17). Remaining: 998
Expanded X to (12,14)-(19,17). Remaining: 996
Expanded Y to (12,13)-(19,18). Remaining: 977
Expanded Y to (12,12)-(19,19). Remaining: 970
Expanded X to (11,12)-(20,19). Remaining: 947
Expanded X to (10,12)-(21,19). Remaining: 941
Expanded Y to (10,11)-(21,20). Remaining: 910
Expanded Y to (10,10)-(21,21). Remaining: 899
Expanded X to (9,10)-(22,21). Remaining: 864
Expanded X to (8,10)-(23,21). Remaining: 854
Expanded Y to (8,9)-(23,22). Remaining: 811
Expanded Y to (8,8)-(23,23). Remaining: 796
Expanded X to (7,8)-(24,23). Remaining: 749
Expanded X to (6,8)-(25,23). Remaining: 735
Target reached at cell (20, 7) during Expansion Y.
Successfully generated maze_32x32_iterative.csv
